# Análise exploratória

**TODO:**
- (1) Tabela
- (2) Gráfico
- (3) Tabela



Solicitação: "Bom dia Seguem dados que demonstram que a transmissão de GPS dos veículos da linha 309 estava ativa e disponível para a SMTR, porém as viagens de sentido VOLTA não foram apontadas no relatório de pagamento.Favor solicitar reprocessamento destes dias, senão teremos de abrir recurso para mais de 180 viagens".

## Análise das viagens da amostra x apuradas

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [18]:
#!pip install matplotlib
import basedosdados as bd
from datetime import timedelta, datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go


pd.set_option('display.max_rows', 10)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = current_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"

Importar amostra

In [19]:
# Nome do arquivo Excel
caminho_amostra = paths["raw"] / "Relatorio de Viagens Realizadas Sentido Volta - Linha 309 - 31.12.2022 e 01.01.2023.xlsx"
sheet1 = pd.read_excel(caminho_amostra, sheet_name=0)
sheet2 = pd.read_excel(caminho_amostra, sheet_name=1)
amostra = pd.concat([sheet1, sheet2], ignore_index=True)

# Renomear as colunas
colunas_renomeadas = {
    'Data': 'data',
    'Linha': 'servico',
    'Carro': 'id_veiculo',
    'Sentido': 'sentido',
    'Saída ': 'hora_inicio',
    'Chegada': 'hora_fim'
}

amostra.rename(columns=colunas_renomeadas, inplace=True)


# Criar as colunas `datetime_partida` e `datetime_chegada`
amostra['datetime_partida'] = amostra['data'].astype(str) + ' ' + amostra['hora_inicio'].astype(str)
amostra['datetime_chegada'] = amostra['data'].astype(str) + ' ' + amostra['hora_fim'].astype(str)

# Transformar essas colunas em tipo datetime
amostra['datetime_partida'] = pd.to_datetime(amostra['datetime_partida'])
amostra['datetime_chegada'] = pd.to_datetime(amostra['datetime_chegada'])

amostra['id_veiculo'] =  amostra['id_veiculo'].astype('str')

amostra

,data,servico,id_veiculo,sentido,hora_inicio,hora_fim,datetime_partida,datetime_chegada
0,2022-12-31,309,41380,VOLTA,04:10:00,05:08:00,2022-12-31 04:10:00,2022-12-31 05:08:00
1,2022-12-31,309,41125,VOLTA,04:20:00,05:15:00,2022-12-31 04:20:00,2022-12-31 05:15:00
2,2022-12-31,309,41054,VOLTA,04:31:00,05:34:00,2022-12-31 04:31:00,2022-12-31 05:34:00
3,2022-12-31,309,41464,VOLTA,04:41:00,05:48:00,2022-12-31 04:41:00,2022-12-31 05:48:00
4,2022-12-31,309,41310,VOLTA,04:51:00,05:45:00,2022-12-31 04:51:00,2022-12-31 05:45:00
...,...,...,...,...,...,...,...,...
176,2023-01-01,309,41125,VOLTA,18:38:00,20:00:00,2023-01-01 18:38:00,2023-01-01 20:00:00
177,2023-01-01,309,41325,VOLTA,19:00:00,20:25:00,2023-01-01 19:00:00,2023-01-01 20:25:00
178,2023-01-01,309,41147,VOLTA,19:20:00,20:27:00,2023-01-01 19:20:00,2023-01-01 20:27:00
179,2023-01-01,309,41030,VOLTA,19:39:00,20:53:00,2023-01-01 19:39:00,2023-01-01 20:53:00


Verificar se existe sobreposição de viagens na amostra

In [20]:
amostra['status'] = np.nan

# Verificação de sobreposição
for index, row in amostra.iterrows():
    mask = (
        (amostra['id_veiculo'] == row['id_veiculo']) & 
        (amostra['datetime_partida'] <= row['datetime_chegada']) & 
        (amostra['datetime_chegada'] >= row['datetime_partida']) &
        (amostra.index != index)
    )
    
    overlapping_rows = amostra[mask]
    
    if overlapping_rows.shape[0] > 0:
        for overlapping_index in overlapping_rows.index:
            # Se a linha de sobreposição tem um horário de partida idêntico à linha atual
            if amostra.at[overlapping_index, 'datetime_partida'] == row['datetime_partida']:
                # Marcar como "Viagem inválida" se o index da linha sobreposta é maior do que o da linha atual
                if overlapping_index > index:
                    amostra.at[overlapping_index, 'status'] = 'Viagem inválida - sobreposição de viagem'
            # Se a linha de sobreposição começa exatamente quando a linha atual termina
            elif amostra.at[overlapping_index, 'datetime_partida'] == row['datetime_chegada']:
                amostra.at[index, 'status'] = np.nan
            # Se a linha de sobreposição começa antes da linha atual terminar e termina depois da linha atual começar (sobreposição real)
            elif amostra.at[overlapping_index, 'datetime_partida'] < row['datetime_chegada'] and amostra.at[overlapping_index, 'datetime_chegada'] > row['datetime_partida']:
                if overlapping_index > index:
                    amostra.at[overlapping_index, 'status'] = 'Viagem inválida - sobreposição de viagem'


amostra

,data,servico,id_veiculo,sentido,hora_inicio,hora_fim,datetime_partida,datetime_chegada,status
0,2022-12-31,309,41380,VOLTA,04:10:00,05:08:00,2022-12-31 04:10:00,2022-12-31 05:08:00,NaN
1,2022-12-31,309,41125,VOLTA,04:20:00,05:15:00,2022-12-31 04:20:00,2022-12-31 05:15:00,NaN
2,2022-12-31,309,41054,VOLTA,04:31:00,05:34:00,2022-12-31 04:31:00,2022-12-31 05:34:00,NaN
3,2022-12-31,309,41464,VOLTA,04:41:00,05:48:00,2022-12-31 04:41:00,2022-12-31 05:48:00,NaN
4,2022-12-31,309,41310,VOLTA,04:51:00,05:45:00,2022-12-31 04:51:00,2022-12-31 05:45:00,NaN
...,...,...,...,...,...,...,...,...,...
176,2023-01-01,309,41125,VOLTA,18:38:00,20:00:00,2023-01-01 18:38:00,2023-01-01 20:00:00,NaN
177,2023-01-01,309,41325,VOLTA,19:00:00,20:25:00,2023-01-01 19:00:00,2023-01-01 20:25:00,NaN
178,2023-01-01,309,41147,VOLTA,19:20:00,20:27:00,2023-01-01 19:20:00,2023-01-01 20:27:00,NaN
179,2023-01-01,309,41030,VOLTA,19:39:00,20:53:00,2023-01-01 19:39:00,2023-01-01 20:53:00,NaN


In [21]:
# não existe nenhuma viagem sobreposta
amostra['status'].value_counts()

Series([], Name: status, dtype: int64)

In [22]:
datas = amostra['data'].unique().astype(str)
datas = [data[:10] for data in datas]
datas
servico = '309'

q = f"""
       SELECT
         data, id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado = '{servico}'
       """
       
query_servico = bd.read_sql(q, from_file=True)
query_servico.info() 
query_servico = query_servico.sort_values(by='datetime_partida')

# remover a letra do início de id_veiculo
query_servico['id_veiculo'] = query_servico['id_veiculo'].apply(
    lambda x: x[1:] if x[0].isalpha() else x
)

query_servico['data'] = pd.to_datetime(query_servico['data'])
query_servico    

Downloading: 100%|██████████| 157/157 [00:00<00:00, 415.35rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   data               157 non-null    dbdate        
 1   id_veiculo         157 non-null    object        
 2   servico_informado  157 non-null    object        
 3   sentido            157 non-null    object        
 4   datetime_partida   157 non-null    datetime64[ns]
 5   datetime_chegada   157 non-null    datetime64[ns]
dtypes: datetime64[ns](2), dbdate(1), object(3)
memory usage: 7.5+ KB


,data,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
124,2022-12-31,41380,309,I,2022-12-31 05:16:25,2022-12-31 06:12:55
88,2022-12-31,41125,309,I,2022-12-31 05:26:03,2022-12-31 06:23:03
63,2022-12-31,41054,309,I,2022-12-31 05:40:57,2022-12-31 06:38:57
148,2022-12-31,41464,309,I,2022-12-31 05:54:34,2022-12-31 06:56:04
132,2022-12-31,41310,309,I,2022-12-31 05:57:43,2022-12-31 06:55:43
...,...,...,...,...,...,...
48,2023-01-01,41325,309,I,2023-01-01 20:32:07,2023-01-01 21:39:07
21,2023-01-01,41147,309,I,2023-01-01 20:35:23,2023-01-01 21:36:23
10,2023-01-01,41030,309,I,2023-01-01 20:59:36,2023-01-01 22:11:36
45,2023-01-01,41322,309,I,2023-01-01 21:22:40,2023-01-01 22:35:10


In [23]:
amostra[(amostra['data'] == '2022-12-31') & (amostra['id_veiculo'] == '41380')]

,data,servico,id_veiculo,sentido,hora_inicio,hora_fim,datetime_partida,datetime_chegada,status
0,2022-12-31,309,41380,VOLTA,04:10:00,05:08:00,2022-12-31 04:10:00,2022-12-31 05:08:00,NaN
25,2022-12-31,309,41380,VOLTA,06:55:00,08:09:00,2022-12-31 06:55:00,2022-12-31 08:09:00,NaN
50,2022-12-31,309,41380,VOLTA,11:23:00,12:41:00,2022-12-31 11:23:00,2022-12-31 12:41:00,NaN
70,2022-12-31,309,41380,VOLTA,14:41:00,15:45:00,2022-12-31 14:41:00,2022-12-31 15:45:00,NaN


In [24]:
query_servico[(query_servico['data'] == '2022-12-31') & (query_servico['id_veiculo'] == '41380')]

,data,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
124,2022-12-31,41380,309,I,2022-12-31 05:16:25,2022-12-31 06:12:55
122,2022-12-31,41380,309,I,2022-12-31 09:36:25,2022-12-31 10:48:55
135,2022-12-31,41380,309,I,2022-12-31 12:57:54,2022-12-31 14:09:24
119,2022-12-31,41380,309,I,2022-12-31 15:58:54,2022-12-31 17:04:54


### 3. Comparar dados da amostra com os dados das viagens completas

#### 3.1 As viagens da amostra foram encontradas dos dados do serviço?

Esta etapa cruza dados do gabarito com as viagens completas para o serviço 663 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens completas deve estar dentro 
do intervalo de mais ou menos 10 minutos do datetime_partida que consta no gabarito.

In [25]:

intervalo_join = 10 # 10 minutos
query_servico = query_servico[['servico_informado','id_veiculo','sentido','datetime_partida','datetime_chegada']]

# 1. Adicionar uma chave temporária
amostra['tmp_key'] = amostra['id_veiculo'].astype('str')
query_servico['tmp_key'] = query_servico['id_veiculo'].astype('str')

# 2. Fazendo o merge usando a chave temporária
tabela_comparativa = pd.merge(amostra, query_servico, on='tmp_key', suffixes=('_amostra', '_apurada'))

# 3. Filtrar os resultados com base no critério do intervalo de tempo
condition = (tabela_comparativa['datetime_partida_apurada'] >= (tabela_comparativa['datetime_partida_amostra'] - pd.Timedelta(minutes=intervalo_join))) & \
            (tabela_comparativa['datetime_partida_apurada'] <= (tabela_comparativa['datetime_partida_amostra'] + pd.Timedelta(minutes=intervalo_join)))

tabela_comparativa = tabela_comparativa[condition]

# Removendo a chave temporária e outras colunas desnecessárias
tabela_comparativa.drop(columns=['tmp_key'], inplace=True)

# Atualizar a coluna 'status' baseada na condição
tabela_comparativa.loc[tabela_comparativa['id_veiculo_amostra'] == tabela_comparativa['id_veiculo_apurada'], 'status'] = 'O veículo existe e operou na linha indicada pelo recurso'
tabela_comparativa.loc[tabela_comparativa['id_veiculo_amostra'] != tabela_comparativa['id_veiculo_apurada'], 'status'] = 'Viagem encontrada no serviço, mas com veículo diferente'
tabela_comparativa

,data,servico,id_veiculo_amostra,sentido_amostra,hora_inicio,hora_fim,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada


Após o cruzamento dentro de um intervalo de 10 minutos, nenhuma viagem da amostra foi identificada. Ao verificar a tabela `viagem_completa`, foram encontradas apenas viagens no sentido de ida nos dias 31/12/2022 e 01/01/2023:

In [26]:
query_servico['sentido'].unique()

array(['I'], dtype=object)

#### Exemplo:

Ao verificar os shapes planejados para o dia, notou-se que existem 2 shapes de Ida e um shape de volta:

#### Viagem de ida (comparação dos dois shapes)

In [27]:
q = f"""
SELECT
 *
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-12-31" AND servico = '309'
""" 
shape_planejado = bd.read_sql(q, from_file=True)
shape_planejado.info() 
shape_planejado

Downloading: 100%|██████████| 3/3 [00:00<00:00,  7.01rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   data                       3 non-null      dbdate 
 1   tipo_dia                   3 non-null      object 
 2   servico                    3 non-null      object 
 3   vista                      3 non-null      object 
 4   consorcio                  3 non-null      object 
 5   sentido                    3 non-null      object 
 6   distancia_planejada        3 non-null      float64
 7   distancia_total_planejada  3 non-null      float64
 8   inicio_periodo             3 non-null      dbtime 
 9   fim_periodo                3 non-null      dbtime 
 10  trip_id_planejado          3 non-null      object 
 11  trip_id                    3 non-null      object 
 12  shape_id                   3 non-null      object 
 13  shape_id_planejado         3 non-null      object 
 14

,data,tipo_dia,servico,vista,consorcio,sentido,distancia_planejada,distancia_total_planejada,inicio_periodo,fim_periodo,trip_id_planejado,trip_id,shape_id,shape_id_planejado,data_shape,shape,sentido_shape,start_pt,end_pt
0,2022-12-31,Sabado,309,Alvorada ↔ Central,Transcarioca,I,35.868,7327.146,22:00:00,10:00:00,7a05c029-499f-434e-9e05-029fdc4558e3,7a05c029-499f-434e-9e05-029fdc4558e3,773t,773t,2022-12-31,"MULTILINESTRING((-43.36598 -23.00152, -43.3675...",I,POINT(-43.36755 -23.00155),POINT(-43.19192 -22.9045)
1,2022-12-31,Sabado,309,Alvorada ↔ Central,Transcarioca,I,35.868,7327.146,10:00:00,22:00:00,O0309AAA0AIDU01,O0309AAA0AIDU01,O0309AAA0AIDU01,O0309AAA0AIDU01,2022-12-31,LINESTRING(-43.3659682760486 -23.0015126993213...,I,POINT(-43.3659682760486 -23.0015126993213),POINT(-43.1919224253361 -22.9044931043239)
2,2022-12-31,Sabado,309,Alvorada ↔ Central,Transcarioca,V,36.055,7327.146,03:20:00,22:45:00,O0309AAA0AVDU01,O0309AAA0AVDU01,O0309AAA0AVDU01,O0309AAA0AVDU01,2022-12-31,LINESTRING(-43.1919129747216 -22.9044908950373...,V,POINT(-43.1919129747216 -22.9044908950373),POINT(-43.3659585238186 -23.0015125015223)


In [28]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-12-31"
  AND timestamp_gps BETWEEN "2022-12-31T05:16:00"
  AND "2022-12-31T06:12:00"
  AND id_veiculo = "C41380"
  """
registro_status = bd.read_sql(q, from_file=True)
registro_status.info() 

Downloading: 100%|██████████| 224/224 [00:00<00:00, 382.91rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           224 non-null    object        
 1   servico              224 non-null    object        
 2   timestamp_gps        224 non-null    datetime64[ns]
 3   posicao_veiculo_geo  224 non-null    object        
 4   status_viagem        224 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 8.9+ KB


In [29]:
shape_planejado_i = shape_planejado.loc[[0,1]]
create_trip_map(registro_status, shape_planejado_i)

#### Viagem de Volta

In [34]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-12-31"
  AND timestamp_gps BETWEEN "2022-12-31T04:11:00"
  AND "2022-12-31T05:10:00"
  AND id_veiculo = "C41380"
  """
registro_status = bd.read_sql(q, from_file=True)
registro_status.info() 

Downloading: 100%|██████████| 112/112 [00:00<00:00, 299.47rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           112 non-null    object        
 1   servico              112 non-null    object        
 2   timestamp_gps        112 non-null    datetime64[ns]
 3   posicao_veiculo_geo  112 non-null    object        
 4   status_viagem        112 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 4.5+ KB


In [35]:
shape_planejado_v = shape_planejado.loc[[2]]
create_trip_map(registro_status, shape_planejado_v)

As viagens de volta do dia 31/12/2022 que não foram identificada na tabela `viagem_completa`, aparecem na tabela `viagem_conformidade` sem nenhum problema quanto aos percentuais de conformidade de sinais de GPS e do trajeto.

In [33]:
q = f"""
       SELECT
         data, id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
       WHERE
         data = '2022-12-31'
         AND id_veiculo = 'C41380'
       """
conformidade = bd.read_sql(q, from_file=True)
registro_status.info() 
conformidade.sort_values(by='datetime_partida')

Downloading: 100%|██████████| 12/12 [00:00<00:00, 33.99rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           111 non-null    object        
 1   servico              111 non-null    object        
 2   timestamp_gps        111 non-null    datetime64[ns]
 3   posicao_veiculo_geo  111 non-null    object        
 4   status_viagem        111 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 4.5+ KB


,data,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
7,2022-12-31,C41380,309,V,2022-12-31 04:11:55,2022-12-31 05:07:25
6,2022-12-31,C41380,309,I,2022-12-31 05:16:25,2022-12-31 06:12:55
8,2022-12-31,C41380,309,I,2022-12-31 05:16:25,2022-12-31 06:12:55
2,2022-12-31,C41380,309,V,2022-12-31 06:58:55,2022-12-31 08:07:25
1,2022-12-31,C41380,309,I,2022-12-31 09:36:25,2022-12-31 10:48:55
...,...,...,...,...,...,...
0,2022-12-31,C41380,309,I,2022-12-31 12:57:54,2022-12-31 14:09:24
5,2022-12-31,C41380,309,I,2022-12-31 12:57:54,2022-12-31 14:09:24
9,2022-12-31,C41380,309,V,2022-12-31 14:45:54,2022-12-31 15:42:54
10,2022-12-31,C41380,309,I,2022-12-31 15:58:54,2022-12-31 17:04:54


Já as viagens de volta do dia 01/01/2023 não aparecem na tabela de conformidade.

In [37]:
q = f"""
       SELECT
         data, id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
       WHERE
         data = '2023-01-01'
         and servico_informado = '309'
       """
conformidade = bd.read_sql(q, from_file=True)
registro_status.info() 
conformidade.sort_values(by='datetime_partida')

Downloading: 0rows [00:00, ?rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           112 non-null    object        
 1   servico              112 non-null    object        
 2   timestamp_gps        112 non-null    datetime64[ns]
 3   posicao_veiculo_geo  112 non-null    object        
 4   status_viagem        112 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 4.5+ KB


,data,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
